# Functionality

> Only development notebook where the goal is to save the ranking in a csv file only (analysis will be done on evaluation.ipynb notebook"

1. The user provides a prompt containing that ask the LM to rank for n sentences with the ideal sentence based on similarity.

2. Save the response (as dataframe) from the model and extract the rank from the free text generated by the model and save it into the same dataframe as a new column

3. Creating a column that has the correlation for each prompt with the gold rank (here you provide the gold rank)

In [141]:
import re
import pandas as pd
from scipy import stats
from scipy.stats import shapiro
import matplotlib.pyplot as plt
import statistics
import ipywidgets as widgets
import os
import csv
import datetime
import openai
from dotenv import load_dotenv
import ast

# this is saved as environment variable in the system.
openai.api_key = os.getenv("---------")

In [ ]:
# gold_rank_example = widgets.Dropdown(options=["[2, 3, 4, 1, 5]"], value="[2, 3, 4, 1, 5]", description='Gold Rank Example', layout={'height': 'auto', "width":"auto"})
# display(gold_rank_example)

insert_gold_rank_manually = widgets.Textarea(placeholder='Enter Gold Rank in the format of this [1,2,3,4,5]', value='', description='Gold Rank', rows=1, layout={'height': 'auto', "width":"auto"})
display(insert_gold_rank_manually)


prompt_identifier = widgets.Dropdown(options=["cot", "zero-shot", "one-shot-1","one-shot-2", "one-shot-3" , "two-shot"], value=None, description='Prompt Type Identifier', layout={'height': 'auto', "width":"auto"})
display(prompt_identifier)

text_input = widgets.Textarea(placeholder='Prompt', value='', description='Enter text:', rows=5, layout={'height': 'auto', "width":"auto"})
display(text_input)


# Creating widgets for the hyperparameters
model_input = widgets.Dropdown(options=["gpt-3.5-turbo", "gpt-4", "text-davinci-003"], value='gpt-3.5-turbo', description='model:', layout={'height': 'auto', "width":"auto"})
display(model_input)
temp_input = widgets.FloatSlider(value=0, min=0, max=1, step=0.05, description='temperature:', layout={'height': 'auto', "width":"auto"})
display(temp_input)
max_tokens_input = widgets.IntSlider(value=500, min=1, max=4000, step=1, description='max_tokens:', layout={'height': 'auto', "width":"auto"})
display(max_tokens_input)
top_p_input = widgets.FloatSlider(value=1.0, min=0, max=1, step=0.05, description='top_p:', layout={'height': 'auto', "width":"auto"})
display(top_p_input)
# freq_penalty_input = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.05, description='frequency_penalty:', layout={'height': 'auto', "width":"auto"})
# display(freq_penalty_input)
# presence_penalty_input = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.05, description='presence_penalty:', layout={'height': 'auto', "width":"auto"})
# display(presence_penalty_input)
# Creating the csv file if it does not exist and adding the new column "Hyperparameters used"


if not os.path.exists('response_eco.csv'):
    with open("response_eco.csv", "a", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Prompt With Task",
                         "Gold Rank",
                         "Prompt Type",
                         "Complete Response from Model",
                         "Output Only From Model",
                         "Hyperparameters used",
                         "Date and Time",
                        "Model"])

# Creating the function that will be called on button click

def on_submit(b):
    entered_text = text_input.value
    
    if model_input.value == "gpt-4" or model_input.value == "gpt-3.5-turbo":
        
        response = openai.ChatCompletion.create(
            model=model_input.value,
            messages=[{"role": "user", "content": entered_text}],
            # prompt=entered_text,
            temperature=temp_input.value,
            max_tokens=max_tokens_input.value,
            top_p=top_p_input.value,
            # frequency_penalty=freq_penalty_input.value,
            # presence_penalty=presence_penalty_input.value,
        )
        
        print("--------------------------------------------------------------------------------")
        print("Input: {}\nOutput: \n \n{}\n".format(entered_text, response['choices'][0]['message']['content']))

    elif model_input.value == "text-davinci-003":
        
        response = openai.Completion.create(
            model=model_input.value,
            prompt=entered_text,
            temperature=temp_input.value,
            max_tokens=max_tokens_input.value,
            top_p=top_p_input.value,
            # frequency_penalty=freq_penalty_input.value,
            # presence_penalty=presence_penalty_input.value,
        )
        print("--------------------------------------------------------------------------------")
        print("Input: {}\nOutput: \n \n{}\n".format(entered_text, response['choices'][0]["text"]))
        
    now = datetime.datetime.now()
    # Storing the hyperparameters used in the response
    params = {
        'model': model_input.value, 
        'temperature': temp_input.value, 
        'max_tokens': max_tokens_input.value, 
        'top_p': top_p_input.value, 
        # 'frequency_penalty': freq_penalty_input.value, 
        # 'presence_penalty': presence_penalty_input.value
    }
    
    
    if model_input.value == "gpt-4" or model_input.value == "gpt-3.5-turbo":
        print("I am in gpt-3.5-turbo")
        data = [
            entered_text,
            insert_gold_rank_manually.value,
            prompt_identifier.value,
            response,
            response['choices'][0]['message']['content'],
            params,
            now.strftime("%Y-%m-%d %H:%M:%S"),
            model_input.value
               ]
        
    elif model_input.value == "text-davinci-003":
        data = [
            entered_text,
            insert_gold_rank_manually.value,
            prompt_identifier.value,
            response,
            response['choices'][0]["text"], # for text-davinci-003
            # response['choices'][0]['message']['content'],
            params,
            now.strftime("%Y-%m-%d %H:%M:%S"),
            model_input.value
               ]
        
    # Open the CSV file for writing
    with open("response_eco.csv", "a", newline='') as file:
        writer = csv.writer(file)
        # Write the data rows
        writer.writerow(data)


submit_button = widgets.Button(description="Submit")
submit_button.on_click(on_submit)
display(submit_button)

# gold rank
# [2, 3, 4, 1, 5]

In [ ]:
df = pd.read_csv("response_eco.csv", encoding='cp1252')
df

In [ ]:
print(df.iloc[-1]['Output Only From Model'])

In [ ]:
'''
Functionality:

Extract the rank from the model's output (which is saved in the csv file)
and create a new column containing (in the same df) the ranks in a list.
'''

# Define a function to extract the numbers associated with Rank
def extract_ranks(text):
    return [int(x) for x in re.findall(r'SAMPLE_TEXT_\d+: Rank (\d+)', text)]

# Apply the function to the column of the dataframe
df['model ranks'] = df['Output Only From Model'].apply(extract_ranks)
df

In [ ]:
'''
Functionality: converting the gold rank that is in string format
now to a list of integer

(If I have pre-processed it while saving the gold-rank then I would not
have to do it. With string you cannot calcualte spearman rank correlation)

'''


def str_to_list(str_list):
    # Remove the brackets from the string
    input_str = str_list[1:-1]

    # Split the string at the commas
    input_list = input_str.split(",")

    # Convert each element to an integer
    input_list = [int(x) for x in input_list]

    return input_list

df['Gold Rank'] = df['Gold Rank'].apply(str_to_list)
type(df.iloc[-1]['Gold Rank'])

In [ ]:
from scipy.stats import spearmanr
'''
For all the rank we got and create a another column with it
'''
def calc_spearman_rank_corr(row):
    gold_rank = row['Gold Rank']
    model_ranks = row['model ranks']
    corr, pvalue = spearmanr(gold_rank, model_ranks)
    return corr

df['spearman_rank_correlation'] = df.apply(calc_spearman_rank_corr, axis=1)
df

In [186]:
'''
Functionarlity:

Saving the final dataframe with all the ranks extracted and the spearman
rank correlation FINALLY!
'''

df.to_csv('complete_response_with_spearman.csv', index=False)

In [187]:
sample_length = len(df)
average_correlation = statistics.mean(df['spearman_rank_correlation'])
variance = statistics.variance(df['spearman_rank_correlation'])
standard_deviation = statistics.stdev(df['spearman_rank_correlation'])
                                      
print("+++++++++++++++++++++++++++++")
print(f"Number of Samples: {sample_length}\nAverage Correlation: {average_correlation:.4f}\nVariance: {variance:.4f}\nStandardDeviation: {standard_deviation:.4f}")
print("+++++++++++++++++++++++++++++")

+++++++++++++++++++++++++++++
Number of Samples: 18
Average Correlation: 0.2722
Variance: 0.2657
StandardDeviation: 0.5154
+++++++++++++++++++++++++++++


# Extra Code

In [ ]:
"""USE THIS IF THE OUTPUT IS LIKE SHOWN BELOW TO EXTRACT THE RANKS: ACTUALLY YOU DO NOT
need this if you write this in the zero shot prompt
=========================================
Provide the rank in the following format

SAMPLE_TEXT_X: Rank X
==========================================
"""

# define the function to apply
def extract_numbers_from_template_one_shot_1_to_cot(text):
    '''
    This is exactly for "zero-shot" template because the output of the model is different like this
    
    1. SAMPLE_TEXT_4 
    2. SAMPLE_TEXT_1 
    3. SAMPLE_TEXT_6 
    4. SAMPLE_TEXT_2 
    5. SAMPLE_TEXT_5 
    6. SAMPLE_TEXT_3
    '''
    pattern = r"(?:Sample Text (\d+)|SAMPLE_TEXT_(\d+))$"
    numbers = re.findall(pattern, text)

    # Convert strings to integers
    numbers = [int(num1 or num2) for num1, num2 in re.findall(pattern, text, re.MULTILINE)]


    return numbers


df['model ranks'] = df.apply(lambda x: extract_numbers_from_template_one_shot_1_to_cot(x['Output']) if x['Prompt']=='zero-shot' else x['model ranks'], axis=1)
df